# SVG Repair Colab Demo\n
Runtime setup + model downloads (SAM2.1-B, YOLO26, GroundingDINO, Big-LaMa) + UI launch.

In [ ]:
# --- REPOSITORY SYNC & AUTORELOAD ---
# Run this cell whenever you want to pull the latest code from VS Code/GitHub.
# Last synced at: 2026-02-09

%load_ext autoreload
%autoreload 2

import os
from pathlib import Path

REPO_URL = "https://github.com/LIdro/svg-colab.git"
REPO_DIR = "/content/svg-colab"
BRANCH = "main"

if not os.path.exists(REPO_DIR):
    print(f"Cloning {REPO_URL}...")
    !git clone {REPO_URL} {REPO_DIR}
else:
    print(f"Syncing with {BRANCH} branch...")
    !git -C {REPO_DIR} fetch origin
    !git -C {REPO_DIR} reset --hard origin/{BRANCH}
    !git -C {REPO_DIR} clean -fd

os.chdir(REPO_DIR)
print(f"Working directory: {os.getcwd()}")
!git status -sb


In [ ]:
!nvidia-smi || true
import torch
print('torch.cuda.is_available =', torch.cuda.is_available())


In [ ]:
# Install dependencies
!sed -i '/pypotrace/d' colab_demo/services/requirements-colab.txt
!pip install -r colab_demo/services/requirements-colab.txt


In [ ]:
import os

# Read secrets from Colab Secrets panel if available:
# - OPENROUTER_API_KEY
# - HF_TOKEN (optional)
# - SAM2_URL (optional direct URL override for sam2.1_b.pt)
# - BIG_LAMA_URL (optional direct URL override for big-lama.pt)
try:
    from google.colab import userdata  # type: ignore
except Exception:
    userdata = None

def read_secret(name: str):
    if userdata is None:
        return os.environ.get(name)
    try:
        return userdata.get(name)
    except Exception:
        return os.environ.get(name)

OPENROUTER_API_KEY = read_secret('OPENROUTER_API_KEY')
HF_TOKEN = read_secret('HF_TOKEN') or read_secret('HUGGINGFACE_TOKEN')
SAM2_URL = read_secret('SAM2_URL')
BIG_LAMA_URL = read_secret('BIG_LAMA_URL')

if OPENROUTER_API_KEY:
    os.environ['OPENROUTER_API_KEY'] = OPENROUTER_API_KEY
if SAM2_URL:
    os.environ['SAM2_URL'] = SAM2_URL
if BIG_LAMA_URL:
    os.environ['BIG_LAMA_URL'] = BIG_LAMA_URL

print('OPENROUTER_API_KEY loaded:', bool(OPENROUTER_API_KEY))
print('HF_TOKEN loaded:', bool(HF_TOKEN))
print('SAM2_URL provided:', bool(SAM2_URL))
print('BIG_LAMA_URL provided:', bool(BIG_LAMA_URL))


In [ ]:
import os
import shutil
from pathlib import Path
import requests
from huggingface_hub import hf_hub_download

ROOT = Path('/content/svg-colab')
WEIGHTS_DIR = ROOT / '.colab_models'
WEIGHTS_DIR.mkdir(parents=True, exist_ok=True)

def ensure_http_download(url: str, dst: Path):
    if dst.exists() and dst.stat().st_size > 0:
        print('[skip] exists:', dst)
        return dst
    print('[download]', url, '->', dst)
    with requests.get(url, stream=True, timeout=120) as r:
        r.raise_for_status()
        with open(dst, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 1024):
                if chunk:
                    f.write(chunk)
    return dst

def try_hf(repo_id: str, filename: str, out_name: str, token=None):
    out_path = WEIGHTS_DIR / out_name
    if out_path.exists() and out_path.stat().st_size > 0:
        print('[skip] exists:', out_path)
        return out_path
    try:
        p = hf_hub_download(repo_id=repo_id, filename=filename, token=token, local_dir=str(WEIGHTS_DIR))
        shutil.copy2(p, out_path)
        print('[ok] HF:', repo_id, filename, '->', out_path)
        return out_path
    except Exception as e:
        print('[warn] HF download failed:', repo_id, filename, e)
        return None

# YOLO26 weights
YOLO26L = try_hf('openvision/yoloe-26l-seg', 'model.pt', 'yoloe-26l-seg.pt', token=HF_TOKEN)
YOLO26X = try_hf('openvision/yoloe-26x-seg', 'model.pt', 'yoloe-26x-seg.pt', token=HF_TOKEN)

# SAM2.1-B weight (same default used in svg-repair: sam2.1_b.pt)
SAM2 = WEIGHTS_DIR / 'sam2.1_b.pt'
if not SAM2.exists() and os.environ.get('SAM2_URL'):
    try:
        ensure_http_download(os.environ['SAM2_URL'], SAM2)
    except Exception as e:
        print('[warn] SAM2_URL download failed:', e)

if not SAM2.exists():
    local_repo_sam = ROOT / 'svg-repair' / 'sam2.1_b.pt'
    if local_repo_sam.exists():
        shutil.copy2(local_repo_sam, SAM2)
        print('[ok] copied SAM2 from repo file:', local_repo_sam)

if not SAM2.exists():
    # Best-effort public URLs (may change; SAM2_URL secret is most reliable)
    sam_urls = [
        'https://github.com/ultralytics/assets/releases/download/v8.3.0/sam2.1_b.pt',
        'https://github.com/ultralytics/assets/releases/download/v8.3.0/sam2_b.pt',
    ]
    for url in sam_urls:
        try:
            ensure_http_download(url, SAM2)
            break
        except Exception as e:
            print('[warn] SAM2 URL failed:', url, e)

# GroundingDINO config + weights
GDINO_CFG = WEIGHTS_DIR / 'groundingdino_swint_ogc.cfg.py'
GDINO_PTH = WEIGHTS_DIR / 'groundingdino_swint_ogc.pth'
ensure_http_download('https://raw.githubusercontent.com/IDEA-Research/GroundingDINO/main/groundingdino/config/GroundingDINO_SwinT_OGC.py', GDINO_CFG)
ensure_http_download('https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth', GDINO_PTH)

# Big-LaMa checkpoint (tries direct URL override first, then HF candidates)
BIG_LAMA = WEIGHTS_DIR / 'big-lama.pt'
if not BIG_LAMA.exists() and os.environ.get('BIG_LAMA_URL'):
    try:
        ensure_http_download(os.environ['BIG_LAMA_URL'], BIG_LAMA)
    except Exception as e:
        print('[warn] BIG_LAMA_URL download failed:', e)

if not BIG_LAMA.exists():
    for repo_id, fname in [
        ('fashn-ai/lama', 'big-lama.pt'),
        ('CodeGoat24/Big-Lama', 'big-lama.pt'),
    ]:
        p = try_hf(repo_id, fname, 'big-lama.pt', token=HF_TOKEN)
        if p and p.exists():
            BIG_LAMA = p
            break

# Export runtime env vars used by the Colab API
if Path(SAM2).exists():
    os.environ['SAM_WEIGHTS'] = str(SAM2)
if YOLO26L and Path(YOLO26L).exists():
    os.environ['YOLO26L_WEIGHTS'] = str(YOLO26L)
if YOLO26X and Path(YOLO26X).exists():
    os.environ['YOLO26X_WEIGHTS'] = str(YOLO26X)
os.environ['GDINO_CONFIG'] = str(GDINO_CFG)
os.environ['GDINO_WEIGHTS'] = str(GDINO_PTH)

if Path(BIG_LAMA).exists():
    os.environ['BIG_LAMA_CHECKPOINT'] = str(BIG_LAMA)
    os.environ['BIG_LAMA_COMMAND'] = 'python /content/svg-colab/colab_demo/services/inpaint_big_lama.py --image {image} --mask {mask} --output {output}'

# Optional Qualcomm LaMa command (weights handled by qai_hub_models at runtime)
os.environ['QUALCOMM_LAMA_DILATED_COMMAND'] = 'python /content/svg-colab/colab_demo/services/inpaint_qualcomm_lama_dilated.py --image {image} --mask {mask} --output {output}'

print('SAM_WEIGHTS =', os.environ.get('SAM_WEIGHTS'))
print('YOLO26L_WEIGHTS =', os.environ.get('YOLO26L_WEIGHTS'))
print('YOLO26X_WEIGHTS =', os.environ.get('YOLO26X_WEIGHTS'))
print('GDINO_CONFIG =', os.environ.get('GDINO_CONFIG'))
print('GDINO_WEIGHTS =', os.environ.get('GDINO_WEIGHTS'))
print('BIG_LAMA_CHECKPOINT exists =', Path(os.environ.get('BIG_LAMA_CHECKPOINT', '/tmp/missing')).exists())
print('OPENROUTER_API_KEY loaded =', bool(os.environ.get('OPENROUTER_API_KEY')))

In [ ]:
%cd /content/svg-colab
# Run uvicorn in the background using nohup and redirect output to a log file
!nohup python -m uvicorn colab_demo.services.api:app --host 0.0.0.0 --port 5700 > uvicorn.log 2>&1 &
print("Uvicorn server started in the background. Check 'uvicorn.log' for output.")
# Give the server a few seconds to fully initialize before the UI attempts to connect
import time
time.sleep(5)

In [ ]:
# Run in another Colab tab/session after API is running
%cd /content/svg-colab
!python colab_demo/ui/gradio_app.py


In [ ]:
import os

# Navigate to the repository directory
%cd /content/svg-colab

# Modify the gradio_app.py file to remove the unsupported 'language="xml"' argument
!sed -i 's/svg_code = gr.Code(label="SVG Code", language="xml")/svg_code = gr.Code(label="SVG Code")/' colab_demo/ui/gradio_app.py

print("Modified gradio_app.py to remove unsupported language argument.")

In [ ]:
# --- SYNC BACK TO GITHUB ---
# Run this cell to push any fixes you made in Colab back to VS Code.

import os

# Update these if your Colab save path is different
COLAB_NOTEBOOK_PATH = "/content/svg-colab/colab_demo/notebook/svg_repair_colab_demo.ipynb"

def push_to_github(message="Fix from Colab"):
    print("Committing changes...")
    !git add {COLAB_NOTEBOOK_PATH}
    !git commit -m "{message}"
    print("Pushing to GitHub...")
    !git push origin main

# push_to_github("Resolved autoreload error")
